In [1]:
# get boilerplate
import json
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
import re

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report


# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

print(sys.version)

#make the depreciation warnings go away
import warnings
#I'm tired of the warnings on functions the professor asks us to use :) 
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning)

3.5.2 |Anaconda 4.1.1 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
### Random acts of Pizza
### from https://www.reddit.com/r/Random_Acts_Of_Pizza/
### Purpose of Random Acts of Pizza:
###
###The Original Random Pizza Delivery Service
### We are open for active Redditors that have not engaged in questionable online 
### behavior (relax, we don't know about all of it, you're probably fine). See The 
### Pizza Library for everything you need to know to get started.
### https://www.reddit.com/r/Random_Acts_Of_Pizza/wiki/index

In [3]:
### Exploratory data analysis

In [51]:
with open('train.json') as json_data:
    d = json.load(json_data)
    

In [5]:
len(d)

4040

In [6]:
#grab a practice set
prac = d[:20]

In [7]:
prac[0].keys()

dict_keys(['requester_subreddits_at_request', 'requester_number_of_posts_on_raop_at_request', 'request_title', 'requester_number_of_posts_on_raop_at_retrieval', 'post_was_edited', 'requester_upvotes_minus_downvotes_at_retrieval', 'requester_number_of_comments_at_retrieval', 'unix_timestamp_of_request_utc', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'request_id', 'requester_username', 'requester_number_of_subreddits_at_request', 'unix_timestamp_of_request', 'giver_username_if_known', 'requester_number_of_comments_in_raop_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'requester_number_of_posts_at_request', 'request_text_edit_aware', 'requester_upvotes_plus_downvotes_at_request', 'request_number_of_comments_at_retrieval', 'requester_number_of_posts_at_retrieval', 'requester_received_pizza', 'requester_days_since_first_post_on_raop_at_request', 'requester_number_of_comments_at_re

In [8]:
prac[0]

{'giver_username_if_known': 'N/A',
 'number_of_downvotes_of_request_at_retrieval': 0,
 'number_of_upvotes_of_request_at_retrieval': 1,
 'post_was_edited': False,
 'request_id': 't3_l25d7',
 'request_number_of_comments_at_retrieval': 0,
 'request_text': 'Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give is greatly appreciated',
 'request_text_edit_aware': 'Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give is greatly appreciated',
 'request_title': 'Request Colorado Springs Help Us Please',
 'requester_accoun

In [66]:
df = pd.read_json('train.json')

In [67]:
df.head()

,giver_username_if_known,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,post_was_edited,request_id,request_number_of_comments_at_retrieval,request_text,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,...,requester_received_pizza,requester_subreddits_at_request,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,requester_user_flair,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc
0,N/A,0,1,0,t3_l25d7,0,Hi I am in need of food for my 4 children we a...,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,0.000000,...,False,[],0,1,0,1,None,nickylvst,1317852607,1317849007
1,N/A,2,5,0,t3_rcb83,0,I spent the last money I had on gas today. Im ...,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",501.111100,...,False,"[AskReddit, Eve, IAmA, MontereyBay, RandomKind...",34,4258,116,11168,None,fohacidal,1332652424,1332648824
2,N/A,0,3,0,t3_lpu5j,0,My girlfriend decided it would be a good idea ...,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",0.000000,...,False,[],0,3,0,3,None,jacquibatman7,1319650094,1319646494
3,N/A,0,1,1,t3_mxvj3,4,"It's cold, I'n hungry, and to be completely ho...","It's cold, I'n hungry, and to be completely ho...","[Request] In Canada (Ontario), just got home f...",6.518438,...,False,"[AskReddit, DJs, IAmA, Random_Acts_Of_Pizza]",54,59,76,81,None,4on_the_floor,1322855434,1322855434
4,N/A,6,6,0,t3_1i6486,5,hey guys:\n I love this sub. I think it's grea...,hey guys:\n I love this sub. I think it's grea...,[Request] Old friend coming to visit. Would LO...,162.063252,...,False,"[GayBrosWeightLoss, RandomActsOfCookies, Rando...",1121,1225,1733,1887,None,Futuredogwalker,1373657691,1373654091


In [22]:
df.columns

Index(['giver_username_if_known', 'request_id', 'request_text_edit_aware',
       'request_title', 'requester_account_age_in_days_at_request',
       'requester_days_since_first_post_on_raop_at_request',
       'requester_number_of_comments_at_request',
       'requester_number_of_comments_in_raop_at_request',
       'requester_number_of_posts_at_request',
       'requester_number_of_posts_on_raop_at_request',
       'requester_number_of_subreddits_at_request',
       'requester_subreddits_at_request',
       'requester_upvotes_minus_downvotes_at_request',
       'requester_upvotes_plus_downvotes_at_request', 'requester_username',
       'unix_timestamp_of_request', 'unix_timestamp_of_request_utc'],
      dtype='object')

In [24]:
prac[0]['requester_received_pizza']

False

In [28]:
prac[4]['requester_received_pizza']

False

In [39]:
### get the labels from the dictionaries
labs = []
for req in prac:
    if req['requester_received_pizza'] == True: print(1)
    else: print(0)
labs = [1 if x['requester_received_pizza'] == True else 0 for x in prac]

0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
1
0
1
0


In [40]:
labs

[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0]

In [12]:
prac[5]

{'giver_username_if_known': 'N/A',
 'number_of_downvotes_of_request_at_retrieval': 3,
 'number_of_upvotes_of_request_at_retrieval': 4,
 'post_was_edited': False,
 'request_id': 't3_14gmeb',
 'request_number_of_comments_at_retrieval': 0,
 'request_text': "Feeling under the weather so I called out off work today! I hate requesting because I feel like I'm begging so I thought I'd give back! \n\n(I'd offer pizza if today were payday :C)",
 'request_text_edit_aware': "Feeling under the weather so I called out off work today! I hate requesting because I feel like I'm begging so I thought I'd give back! \n\n(I'd offer pizza if today were payday :C)",
 'request_title': "[REQUEST] I'll give a two week xbox live code for a slice of pie!",
 'requester_account_age_in_days_at_request': 582.7765856481482,
 'requester_account_age_in_days_at_retrieval': 946.2708796296296,
 'requester_days_since_first_post_on_raop_at_request': 340.8193287037037,
 'requester_days_since_first_post_on_raop_at_retrieval': 

In [13]:
### maybe requests have to do with the time of day
import datetime
print(
    datetime.datetime.fromtimestamp(
        int(1354911700.0)
    ).strftime('%Y-%m-%d %H:%M:%S')
)


2012-12-07 12:21:40


In [34]:
### get human readible times

def day_time(x):
    y = ''
    if datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 10:
        y = 'morning'
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 10 & datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 16:
        y = 'midday'
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 16 & datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 21:
        y = 'evening'
    else: 
        y = 'night'
    return y
        
def human_time(a):
    import datetime
    from datetime import date
    import calendar
    ### for the data in raop, return human time.  maybe the time of day matters
    a['human_readable_local_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).strftime('%Y-%m-%d %H:%M:%S')
    a['human_readable_UTC_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request_utc']).strftime('%Y-%m-%d %H:%M:%S')
    #a['weekday'] = date(fromtimestamp(a['human_readable_UTC_time'])).weekday()
    a['weekday'] = calendar.day_name[datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).weekday()]
    a['time_of_date'] = day_time(a)
    return a

prac1 = [human_time(x) for x in prac]

prac1[0]
    

{'giver_username_if_known': 'N/A',
 'human_readable_UTC_time': '2011-10-05 14:10:07',
 'human_readable_local_time': '2011-10-05 15:10:07',
 'number_of_downvotes_of_request_at_retrieval': 0,
 'number_of_upvotes_of_request_at_retrieval': 1,
 'post_was_edited': False,
 'request_id': 't3_l25d7',
 'request_number_of_comments_at_retrieval': 0,
 'request_text': 'Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give is greatly appreciated',
 'request_text_edit_aware': 'Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give 

In [36]:
prac1[1]

{'giver_username_if_known': 'N/A',
 'human_readable_UTC_time': '2012-03-24 21:13:44',
 'human_readable_local_time': '2012-03-24 22:13:44',
 'number_of_downvotes_of_request_at_retrieval': 2,
 'number_of_upvotes_of_request_at_retrieval': 5,
 'post_was_edited': False,
 'request_id': 't3_rcb83',
 'request_number_of_comments_at_retrieval': 0,
 'request_text': 'I spent the last money I had on gas today. Im broke until next Thursday :(',
 'request_text_edit_aware': 'I spent the last money I had on gas today. Im broke until next Thursday :(',
 'request_title': '[Request] California, No cash and I could use some dinner',
 'requester_account_age_in_days_at_request': 501.11109953703703,
 'requester_account_age_in_days_at_retrieval': 1122.279837962963,
 'requester_days_since_first_post_on_raop_at_request': 0.0,
 'requester_days_since_first_post_on_raop_at_retrieval': 621.1270717592593,
 'requester_number_of_comments_at_request': 0,
 'requester_number_of_comments_at_retrieval': 1000,
 'requester_nu

In [37]:
len(prac1)

20

In [40]:
len(labs)

20

In [42]:
labs

[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0]

In [43]:
prac1[5]

{'giver_username_if_known': 'N/A',
 'human_readable_UTC_time': '2012-12-07 12:21:40',
 'human_readable_local_time': '2012-12-07 12:21:40',
 'number_of_downvotes_of_request_at_retrieval': 3,
 'number_of_upvotes_of_request_at_retrieval': 4,
 'post_was_edited': False,
 'request_id': 't3_14gmeb',
 'request_number_of_comments_at_retrieval': 0,
 'request_text': "Feeling under the weather so I called out off work today! I hate requesting because I feel like I'm begging so I thought I'd give back! \n\n(I'd offer pizza if today were payday :C)",
 'request_text_edit_aware': "Feeling under the weather so I called out off work today! I hate requesting because I feel like I'm begging so I thought I'd give back! \n\n(I'd offer pizza if today were payday :C)",
 'request_title': "[REQUEST] I'll give a two week xbox live code for a slice of pie!",
 'requester_account_age_in_days_at_request': 582.7765856481482,
 'requester_account_age_in_days_at_retrieval': 946.2708796296296,
 'requester_days_since_firs

In [44]:
prac1[9]

{'giver_username_if_known': 'N/A',
 'human_readable_UTC_time': '2011-08-31 13:59:42',
 'human_readable_local_time': '2011-08-31 14:59:42',
 'number_of_downvotes_of_request_at_retrieval': 0,
 'number_of_upvotes_of_request_at_retrieval': 6,
 'post_was_edited': True,
 'request_id': 't3_k0l9j',
 'request_number_of_comments_at_retrieval': 21,
 'request_text': "Austin, Texas\n\nMy two roommates and I are hungry as hell. We were all sort of counting on the deposit from our last apartment to help us out, but they claimed there was damages, so we did not receive anything. So, we're sort of struggling. Is anyone able to help us out for dinner tonight? We'd really appreciate it!\n\nEDIT: Received a Dominos gift card from jetboyterp! THANK YOU!",
 'request_text_edit_aware': "Austin, Texas\n\nMy two roommates and I are hungry as hell. We were all sort of counting on the deposit from our last apartment to help us out, but they claimed there was damages, so we did not receive anything. So, we're sort

In [45]:
prac1[10]

{'giver_username_if_known': 'N/A',
 'human_readable_UTC_time': '2013-01-18 17:34:46',
 'human_readable_local_time': '2013-01-18 17:34:46',
 'number_of_downvotes_of_request_at_retrieval': 4,
 'number_of_upvotes_of_request_at_retrieval': 5,
 'post_was_edited': False,
 'request_id': 't3_16upcl',
 'request_number_of_comments_at_retrieval': 14,
 'request_text': "I've been unemployed but working odd jobs. I worked a job yesterday which I thought paid up front - it did not. Now I'm sitting at home dreaming of pizza (literally, I took a nap and dreamt there was pizza in my fridge, but woke up to nothing but leftover pasta). Will pay forward when I get a chance!",
 'request_text_edit_aware': "I've been unemployed but working odd jobs. I worked a job yesterday which I thought paid up front - it did not. Now I'm sitting at home dreaming of pizza (literally, I took a nap and dreamt there was pizza in my fridge, but woke up to nothing but leftover pasta). Will pay forward when I get a chance!",
 'r

In [46]:
prac1[16]

{'giver_username_if_known': 'N/A',
 'human_readable_UTC_time': '2012-11-24 22:31:33',
 'human_readable_local_time': '2012-11-24 22:31:33',
 'number_of_downvotes_of_request_at_retrieval': 2,
 'number_of_upvotes_of_request_at_retrieval': 4,
 'post_was_edited': False,
 'request_id': 't3_13r18s',
 'request_number_of_comments_at_retrieval': 3,
 'request_text': 'It would be much appreciated, and payed forward later on. ',
 'request_text_edit_aware': 'It would be much appreciated, and payed forward later on. ',
 'request_title': '[request] Cookeville, TN. My dog recently died and I could use a pizza to cheer my family &amp; me up. ',
 'requester_account_age_in_days_at_request': 66.15856481481481,
 'requester_account_age_in_days_at_retrieval': 442.22925925925927,
 'requester_days_since_first_post_on_raop_at_request': 56.08336805555555,
 'requester_days_since_first_post_on_raop_at_retrieval': 432.1540625,
 'requester_number_of_comments_at_request': 8,
 'requester_number_of_comments_at_retrieval

In [47]:
prac1[18]

{'giver_username_if_known': 'ded_reckoning',
 'human_readable_UTC_time': '2012-09-11 11:41:38',
 'human_readable_local_time': '2012-09-11 12:41:38',
 'number_of_downvotes_of_request_at_retrieval': 1,
 'number_of_upvotes_of_request_at_retrieval': 6,
 'post_was_edited': False,
 'request_id': 't3_zpzly',
 'request_number_of_comments_at_retrieval': 7,
 'request_text': "-I'm located in Virginia. It would be great to have some pizza tonight, feeling under the weather. I don't get paid until the weekend, and this would make our day. ",
 'request_text_edit_aware': "-I'm located in Virginia. It would be great to have some pizza tonight, feeling under the weather. I don't get paid until the weekend, and this would make our day. ",
 'request_title': "[Request] Virginia. Girlfriend and I our sick, don't get paid until Friday, and would love a pizza. Willing to pay it forward.",
 'requester_account_age_in_days_at_request': 183.93306712962962,
 'requester_account_age_in_days_at_retrieval': 634.49731

In [55]:
### We can use these transformations later for a robust ML project, but for now let's 
### extract the original text and if they succeeded in getting a pizza out of the deal
dat = [{k: d[x][k] for k in ('requester_received_pizza', 'request_text')} for x in range(len(d))]

#dat = dict((k, d[k]) for k in ('requester_received_pizza', 'request_text'))

In [54]:
d[0]['request_id']

't3_l25d7'

In [56]:
dat[:5]

[{'request_text': 'Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give is greatly appreciated',
  'requester_received_pizza': False},
 {'request_text': 'I spent the last money I had on gas today. Im broke until next Thursday :(',
  'requester_received_pizza': False},
 {'request_text': "My girlfriend decided it would be a good idea to get off at Perth bus station when she was coming to visit me and has since had to spend all her money on a taxi to get to me here in Dundee. Any chance some kind soul would get us some pizza since we don't have any cash anymore?",
  'requester_received_pizza': False},
 {'request_text': "It's cold, I'n hungry, and to be completely honest I'm broke. My mum said we're having leftovers for dinner. A random pizza arriving w

In [57]:
from sklearn.feature_extraction import DictVectorizer

In [58]:
dat_prac = dat[:15]

In [60]:
v = DictVectorizer(sparse=False)
dat_t = v.fit_transform(dat_prac)

In [62]:
dat_t.shape

(15, 16)

In [63]:
import scipy

In [64]:
scipy.sparse(dat_prac)

TypeError: 'module' object is not callable

In [104]:
## this one works
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df.request_text.tolist())

In [113]:
df.request_text.tolist()[:5]

['Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give is greatly appreciated',
 'I spent the last money I had on gas today. Im broke until next Thursday :(',
 "My girlfriend decided it would be a good idea to get off at Perth bus station when she was coming to visit me and has since had to spend all her money on a taxi to get to me here in Dundee. Any chance some kind soul would get us some pizza since we don't have any cash anymore?",
 "It's cold, I'n hungry, and to be completely honest I'm broke. My mum said we're having leftovers for dinner. A random pizza arriving would be nice.\n\nEdit: We had leftovers.",
 "hey guys:\n I love this sub. I think it's great. (Except the sob stories. I miss when this place was fun!) Anywho, I've given a pizza out

In [106]:
# get labels
Y_train_counts = np.array(df.requester_received_pizza)

In [75]:
Y_train_counts.shape

(4040,)

In [76]:
type(Y_train_counts)

numpy.ndarray

In [77]:
Y_train_counts[:5]

array([False, False, False, False, False], dtype=bool)

In [91]:
b =np.random.shuffle(prac1)

In [114]:
test_data, test_labels = X_train_counts[int(X_train_counts.shape[0]/2):], Y_train_counts[int(Y_train_counts.shape[0]/2):]
dev_data, dev_labels = X_train_counts[int(X_train_counts.shape[0]/2):], Y_train_counts[int(Y_train_counts.shape[0]/2):]


In [108]:
test_data.shape

(2020, 12593)

In [109]:
dev_data.shape

(2020, 12593)

In [115]:
tr = test_data
d = count_vect.fit_transform(test_data)
#part A What is the size of the vocabulary? What is the average number of non-zero
#features per example? What fraction of the entries in the matrix are non-zero?

def nzf(tr):
    #s = d.shape[1]
    avgs = []
    for art in tr:
        nonzero_feat = 0
        for term in v.get_feature_names():
            if term in art:
                #print(term)
                nonzero_feat +=1
            else: #print(term, "not in article")
                continue
        try: avgs.append(nonzero_feat)
        except: 0
    #this is an average of averages, change that
    return np.average(avgs)

def nzf2(tr):
    avgs = []
    for art in tr:
        nonzero_feat = len(set(art.lower().split()))
        avgs.append(nonzero_feat)
    return np.average(avgs)
        
        

def porp(m):
    m1 = m.toarray()
    tot_size = m.shape[0] * m.shape[1]
    x = 0
    for i in range(m.shape[0]):
        x += m.shape[1] - np.count_nonzero(m1[i] == 0)
    return x/tot_size

print("Part A:  The sparse matrix contains", d.shape[1], "words in its vocabulary.")
print("Part A: The average number of non-zero features per example is", nzf2(tr))
print("Part A: What fraction of entries in the matrix are non-zero?", porp(d) )

#part B

#What are the 0th and last feature strings (in alphabetical order)? 
#Hint: use the vectorizer's get_feature_names function.
print("Part B: The first feature in alphabetical order is '%s' and the last is '%s'." 
      % (count_vect.get_feature_names()[0], count_vect.get_feature_names()[-1]))

#part C
#Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. 
#Confirm the training vectors are appropriately shaped. Now what's the average number of 
#non-zero features per example?
c = [False, True]
#c =  ["atheism", "graphics", "space", "religion"]
#c = ['candy', 'dog']
def ptc(trv, voc):
    avgs = []
    for art in trv:
        nonzero_feat = 0
        for word in voc:
            if word in art:
                nonzero_feat += 1
            else: continue
        avgs.append(nonzero_feat)
    return np.average(avgs)

print('Part C: The average number of non-zero features using the parsed down vocabulary of %s is %s.' % (c, ptc(tr, c)))

#part D: Instead of extracting unigram word features, use "analyzer" and "ngram_range" 
#to extract bigram and trigram character features. What size vocabulary does this yield?

cv = CountVectorizer(analyzer='char_wb', ngram_range=(2,3))
dd = cv.fit_transform(tr)
print('Part D: When using bi- and trigrams, the vocabulary has %s features.' % (dd.shape[1]))

#e. Use the "min_df" argument to prune words that appear in fewer than 10 documents.
#What size vocabulary does this yield?

ev = CountVectorizer(min_df= 10)
ed = ev.fit_transform(tr)
print('Part E: When using a a term fequency of at least 10, the vocabulary has %s features.' %(ed.shape[1]))


#f.f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? 
#Hint: build a vocabulary for both train and dev and look at the size of the difference.
v1 = CountVectorizer()

fd = v1.fit_transform(dev_data)
# set theory, find (size(set(dev)) - (size(set(train AND dev)))/size(set(dev))

fl = (len(set(v1.vocabulary_)) - len(set(set(v1.vocabulary_)&set(v.vocabulary_))))/len(set(v1.vocabulary_))
print('Part F: The porportion of development features not found in the training features is %s.' %fl)
#P2()

AttributeError: lower not found

In [214]:
# make the labels as numbers
df['if_pizza'] = df.apply(lambda row: 0 if row['requester_received_pizza'] == False else 1, axis=1)
df['if_pizza'].value_counts()

# for simple random split, you can and should use cross validation/grid search for later model selection
from sklearn.model_selection import StratifiedShuffleSplit

X = df[['request_text', 'request_title']]
y = df['requester_received_pizza']

# a random pipeline
#defines all the steps
#we can add a preprocessing componenet if we wish
#add feature union for final project 
#see http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html
#label encoder when you have lots of classes: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
  #merge text and numbers for models: http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html
clf = Pipeline([
    ('counter', CountVectorizer()),
    ('mnb', MultinomialNB())])

sss = StratifiedShuffleSplit(test_size=0.25, random_state=1)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [210]:
X_train.shape

(3030, 2)

In [211]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [118]:
sss

StratifiedShuffleSplit(n_splits=10, random_state=1, test_size=0.25,
            train_size=None)

In [122]:
sss.get_n_splits

<bound method BaseShuffleSplit.get_n_splits of StratifiedShuffleSplit(n_splits=10, random_state=1, test_size=0.25,
            train_size=None)>

In [149]:
clf1 = Pipeline([
    ('counter', CountVectorizer(analyzer='word', ngram_range=(1,3))),
    ('mnb', MultinomialNB())])

sss1 = StratifiedShuffleSplit(test_size=0.5, random_state=0)
for train_index, test_index in sss1.split(X, y):
    X_train1, X_test1 = X[train_index], X[test_index]
    y_train1, y_test1 = y[train_index], y[test_index]

clf1.fit(X_train1, y_train1)
y_pred1 = clf1.predict(X_test1)
print(classification_report(y_test1, y_pred1))

             precision    recall  f1-score   support

          0       0.75      1.00      0.86      1523
          1       0.20      0.00      0.00       497

avg / total       0.62      0.75      0.65      2020



In [176]:
unique, counts = np.unique(y_pred1, return_counts=True)

In [177]:
counts

array([2015,    5])

In [150]:
###look at cross validation
##http://scikit-learn.org/stable/modules/cross_validation.html

In [178]:
def preprocess(x):
    """Use a series of regex expressions to remove unwanted characters"""
    #remove non-alpha-numeric characters, replace with whitespace
    x1 = re.sub(r'[^a-zA-Z_0-9_\s]'," ", x).lower()
    #replae all numbers with a single token and a space afterwards
    x1a = re.sub(r'[0-9]+', 'number ', x1)
    #x1b = re.sub(r'[_]+', ' ', x1a)
    #even though there are words that are just '_____', f1 actuall decreases when they're removed
    #remove newlines
    x2 = re.sub(r'[\n]', " ", x1a)
    #scrub out extra spaces
    x3 = re.sub(r'\s+', ' ', x2)  #other steps might have added extra space; remove
    return x3.strip()

clf2 = Pipeline([
    ('counter', TfidfVectorizer(preprocessor=preprocess, stop_words='english', norm='l2')),
    ('mnb', MultinomialNB())])

#sss2 = StratifiedShuffleSplit(test_size=0.25, random_state=1)
for train_index, test_index in sss1.split(X, y):
    X_train2, X_test2 = X[train_index], X[test_index]
    y_train2, y_test2 = y[train_index], y[test_index]

clf2.fit(X_train2, y_train2)
y_pred2 = clf2.predict(X_test2)
print(classification_report(y_test2, y_pred2))

             precision    recall  f1-score   support

          0       0.75      1.00      0.86      1523
          1       0.00      0.00      0.00       497

avg / total       0.57      0.75      0.65      2020



/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [168]:
unique, counts = np.unique(y_train2, return_counts=True)

In [169]:
counts

array([2284,  746])

In [179]:
#look at http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py
#or wordclouds https://amueller.github.io/word_cloud/
unique, counts = np.unique(y_pred2, return_counts=True)

In [180]:
counts

array([2020])

In [181]:
unique, counts = np.unique(y_test2, return_counts=True)

In [182]:
counts

array([1523,  497])

In [183]:
df['requester_received_pizza'].value_counts()

False    3046
True      994
Name: requester_received_pizza, dtype: int64

In [215]:
from __future__ import print_function

import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text),
                 'num_sentences': text.count('.')}
                for text in posts]


class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features


pipeline = Pipeline([
    # Extract the subject & body
    #('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for pulling features from the post's subject line
            ('get-title', Pipeline([
                ('selector', ItemSelector(key='request_title')),
                ('tfidf', TfidfVectorizer()),
            ])),

            # Pipeline for standard bag-of-words model for body
            ('get-request', Pipeline([
                ('selector', ItemSelector(key='request_text')),
                ('tfidf', TfidfVectorizer())
            ])),
            
            """('get-number', Pipeline([
                ('selector', ItemSelector(key='number'))
            ]))"""

        ]
    )),

    # Use a SVC classifier on the combined features
    ('nb', MultinomialNB()),
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


'''
# limit the list of categories to make running this example faster.
categories = ['alt.atheism', 'talk.religion.misc']
train = fetch_20newsgroups(random_state=1,
                           subset='train',
                           categories=categories,
                           )
test = fetch_20newsgroups(random_state=1,
                          subset='test',
                          categories=categories,
                          )

pipeline.fit(train.data, train.target)
y = pipeline.predict(test.data)
print(classification_report(y, test.target))
'''

             precision    recall  f1-score   support

          0       0.75      1.00      0.86       762
          1       0.00      0.00      0.00       248

avg / total       0.57      0.75      0.65      1010



/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


"\n# limit the list of categories to make running this example faster.\ncategories = ['alt.atheism', 'talk.religion.misc']\ntrain = fetch_20newsgroups(random_state=1,\n                           subset='train',\n                           categories=categories,\n                           )\ntest = fetch_20newsgroups(random_state=1,\n                          subset='test',\n                          categories=categories,\n                          )\n\npipeline.fit(train.data, train.target)\ny = pipeline.predict(test.data)\nprint(classification_report(y, test.target))\n"

In [216]:
#do this for step 2